Load files

In [ ]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
import os
import numpy as np
import pynapple as nap
import pandas as pd
from pathlib import Path
from ephyviewer import mkQApp, MainViewer, TraceViewer

import os
import warnings
from scipy.stats import zscore
warnings.filterwarnings("ignore")
from ephyviewer import mkQApp, MainViewer, TraceViewer, TimeFreqViewer
from ephyviewer import InMemoryAnalogSignalSource
from ephyviewer import mkQApp, MainViewer, TraceViewer, CsvEpochSource, EpochEncoder
import json
import IPython
from matplotlib import cm
from matplotlib.colors import to_hex
import ast
from ephyviewer import AnalogSignalSourceWithScatter
from IPython.display import display
from ipyfilechooser import FileChooser

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
#######################################################################################
                                # Define functions #
#######################################################################################
def restriction_parameter(All_Spindle):
    nb_spindle = All_Spindle.shape[0]
    listtodrop = []
    for tt in range(nb_spindle-1):
        # merge spdl that starts within a spdl
        if(All_Spindle['end time'][tt]>All_Spindle['start time'][tt + 1]):
            if(All_Spindle['Duration'][tt]<All_Spindle['Duration'][tt + 1]):
                if(All_Spindle['start time'][tt]<All_Spindle['start time'][tt + 1]):
                    All_Spindle['start time'][tt+1] = All_Spindle['start time'][tt]
                    listtodrop.append(tt)
                else:
                    listtodrop.append(tt)
            if(All_Spindle['Duration'][tt]>All_Spindle['Duration'][tt + 1]):
                if(All_Spindle['end time'][tt]<All_Spindle['end time'][tt + 1]):
                    All_Spindle['end time'][tt] = All_Spindle['end time'][tt + 1]
                    listtodrop.append(tt+1)
                else:
                    listtodrop.append(tt+1)
    """
    for tt in range(nb_spindle-1):
        # merge spdls that are 200ms apart
        if((All_Spindle['start time'][tt + 1] - All_Spindle['end time'][tt])<200):
            if((All_Spindle['Duration'][tt])<All_Spindle['Duration'][tt + 1]): #first spdl longer so remove/merge the second one
                All_Spindle['start time'][tt + 1] = min(All_Spindle['start time'][tt], All_Spindle['start time'][tt+1])
                All_Spindle['end time'][tt+ 1] = max(All_Spindle['end time'][tt + 1], All_Spindle['end time'][tt])
                listtodrop.append(tt)
            if((All_Spindle['Duration'][tt+1])<All_Spindle['Duration'][tt]): #second spdl longer so remove/merge the first one
                All_Spindle['start time'][tt] = min(All_Spindle['start time'][tt], All_Spindle['start time'][tt+1])
                All_Spindle['end time'][tt] = max(All_Spindle['end time'][tt + 1], All_Spindle['end time'][tt])
                listtodrop.append(tt+1)
    """
    for tt in range(nb_spindle):
        #Update duration because of the merging
        All_Spindle['Duration'][tt]=All_Spindle['end time'][tt]-All_Spindle['start time'][tt]

    for tt in range(nb_spindle): #All_Spindle.index:
        #Remove Spdl that last less than 500ms
        if (All_Spindle['Duration'][tt]<500):
            listtodrop.append(tt)        
    
    All_Spindle = All_Spindle.drop(listtodrop) 
    All_Spindle = All_Spindle.reset_index(drop=True)
    return All_Spindle


#Load LFP coordinates 
Channels = f'{os.getcwd()}\_LFP_coordinates_of_all_mice.csv'
all_LFPcoordinates = pd.read_csv(Channels, index_col=0)


In [ ]:
try: # tries to retrieve dpath either from a previous run or from a previous notebook
    %store -r dpath
    #dpath=dpath.parent
except:
    print("the path was not defined in store")
    dpath = "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/Analysed2025_AB/"


fc1 = FileChooser(dpath,select_default=True, show_only_dirs = True, title = "<b>Go inside the folder containing the LFP raw file</b>")
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

# Register callback function
fc1.register_callback(update_my_folder)

In [ ]:
folder_base = Path(dpath) #Path(dpath).parent
print(folder_base)        
      
#Load signals

LFPfile = Path(f'{folder_base}\DataFrame_rawdataDS.pkl')
LFPs_df = pd.read_pickle(LFPfile)
samplerate = 1000 
numchannel = LFPs_df.shape[1]
rec_ch_list = LFPs_df.columns.values
# Load LFPs timestamps 
for file_pathTS in folder_base.parent.parent.glob('**/continuous/*/timeStampsDS.npy'):
    print('LFPs timestamps file found')
    LFPtimestamps = np.load(file_pathTS)  
print(round(LFPs_df.shape[0]/samplerate/60), 'min of recording')

# Identify mouse & choose threshold for detection
mouse = []
pos_mice = []
for mouse_name in all_LFPcoordinates.index:
    if mouse_name in LFPfile.__str__():
        mouse.append(mouse_name)
        pos_mice.append(LFPfile.__str__().find(mouse_name)) 
mouse = [x for _, x in sorted(zip(pos_mice, mouse))] # sort mouse in the same order as they appear in the path
mouse=mouse[0]   

# Identify electrodes & create differential LFPs
all_LFPcoordinates = all_LFPcoordinates.astype(str)
for region in all_LFPcoordinates.loc[mouse].index:
    locals()[region] = []
    locals()[f'{region}_ch'] = []
ID=0
RecordedArea=[]
combined0=[]
rec_ch_list_mouse = [value for value in rec_ch_list if 0+(ID*32) <= value <= 31+(ID*32)]
for rec_ch in rec_ch_list_mouse:
    for idx, LFPcoord_str in enumerate(all_LFPcoordinates.loc[mouse]):
        region = all_LFPcoordinates.loc[mouse].index[idx]
        if LFPcoord_str != 'nan':
            LFPcoord = LFPcoord_str.split('_')[:2] # only take into account the 2 first of electrode of that region 
            num_ch = np.where(str(rec_ch-(ID*32)) == np.array(LFPcoord))[0]
            if len(num_ch) > 0:
                region = all_LFPcoordinates.loc[mouse].index[idx]
                LFP = locals()[region]
                LFP = LFP-np.array(LFPs_df[(rec_ch)]) if len(LFP) > 0 else np.array(LFPs_df[(rec_ch)])
                locals()[region] = LFP
                locals()[f'{region}_ch'].append(rec_ch)
                
                break
            continue    
for region in all_LFPcoordinates.loc[mouse].index:
    LFP = locals()[region]
    LFP_ch = locals()[f'{region}_ch']
    if len(LFP) > 0:
        RecordedArea.append(region) 
        combined0 = zscore(LFP[:,np.newaxis]) if len(combined0) == 0 else np.append(combined0, zscore(LFP[:,np.newaxis]), axis=1)

    
# Load Sleep Scoring    
ScoringFile= folder_base / f'Sleep_Scoring_6Stages_5sEpoch.csv'
SleepScored = pd.read_csv(ScoringFile)
SleepScored['label']= SleepScored['label'].str.extract(r'(\d+)', expand=False)
SleepScoredTS=np.array(SleepScored['label'])
scale_factor=samplerate/0.2  #cause scoring was done in 5 seconds bin, ie 0.2 Hz              
SleepScoredTS_upscaled0 = np.repeat(SleepScoredTS, scale_factor, axis=0)
SleepScoredTS_upscaled0=SleepScoredTS_upscaled0.astype(int)
SleepScoredTS_upscaled=SleepScoredTS_upscaled0.copy()

SleepScoredTS_upscaled[SleepScoredTS_upscaled==2]=1 # Transform QW in AW
SleepScoredTS_upscaled[SleepScoredTS_upscaled==4]=1 # Transform REM in AW
SleepScoredTS_upscaled[SleepScoredTS_upscaled==5]=1 # Transform IS in AW
SleepScoredTS_upscaled[SleepScoredTS_upscaled==6]=1 # Transform undefined in AW

if len(CA1) > len(SleepScoredTS_upscaled):
    SleepScoredTS_upscaled = np.pad(SleepScoredTS_upscaled, (0, (len(CA1) - len(SleepScoredTS_upscaled))), constant_values=SleepScoredTS_upscaled[-1])
elif len(CA1) < len(SleepScoredTS_upscaled):
    SleepScoredTS_upscaled=SleepScoredTS_upscaled[:len(CA1)]

notNREMBool = SleepScoredTS_upscaled==1 


CA1=zscore(np.array(CA1))#-np.mean(np.array(CA1)))#/np.mean(np.array(CA1))
S1=zscore(np.array(S1))#-np.mean(np.array(S1)))#/np.mean(np.array(S1))
PFC=zscore(np.array(PFC))#-np.mean(np.array(PFC)))#/np.mean(np.array(PFC))

CA1[notNREMBool] = 0
S1[notNREMBool] = 0
PFC[notNREMBool] = 0

SWR

In [ ]:
lfp = nap.Tsd(t=np.arange(len(CA1))/samplerate, d=CA1)
freqs = np.geomspace(3, 200, 120)
mwt_RUN = nap.compute_wavelet_transform(lfp, fs=samplerate, freqs=freqs, gaussian_width=4, window_length=1) # gw=4, wl=1 only very pretty SWRs
ripple_freq_index = np.logical_and(freqs > 120, freqs < 200)
ripple_power = np.mean(np.abs(mwt_RUN[:, ripple_freq_index]), 1)
smoothed_ripple_power = ripple_power.smooth(0.01) #in seconds 0.005 

In [ ]:
winlen = 10 # default window length in sec

#you must first create a main Qt application (for event loop)
app = mkQApp()

t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer(debug=True, show_auto_scale=True)

#create a viewer for signal
combined = np.stack([zscore(lfp.as_array())/50, smoothed_ripple_power.as_array(),np.linspace(thCA1, thCA1,len(lfp.as_array()) )], axis=1)
source =InMemoryAnalogSignalSource(combined, samplerate, t_start)
view1 = TraceViewer(source=source)

view1.params['xsize']= winlen
view1.params['display_labels'] = True
#view1.params['scale_mode'] = 'same_for_all'
colormap = np.insert(['#88FF88', '#8888FF', '#FF8888']* 10, 0, '#FFFFFF')
view1.by_channel_params[f'ch1', 'color'] = colormap[0]
view1.by_channel_params[f'ch0', 'color'] = colormap[10]
view1.auto_scale()


# FFT
view3 = TimeFreqViewer(source=source, name='FFT')

view3.params['show_axis'] = True
view3.params['timefreq', 'f_start'] = 100
view3.params['timefreq', 'f_stop'] = 200
view3.params['timefreq', 'deltafreq'] = 1 #interval in Hz
view3.params['xsize'] = winlen
view3.by_channel_params[f'ch0', 'clim'] = .0100
view3.by_channel_params[f'ch0', 'visible'] = True

#show main window and run Qapp

win.add_view(view1)
win.add_view(view3)
win.navigation_toolbar.spinbox_xsize.setValue(winlen)
win.show()

app.exec()

# press '1', '2', '3', '4' etc, to encode state.
# or toggle 'Time range selector' and then use 'Insert within range'

In [ ]:
thCA1=.15
threshold_ripple_power = smoothed_ripple_power.threshold(thCA1) # .15 ok, but few for L1 // .25 only very pretty SWRs // .1 too much for TBC
rip_ep = threshold_ripple_power.time_support
rip_ep['dur_ms']=np.round((rip_ep['end']-rip_ep['start'])*1000)
rip_ep=rip_ep[rip_ep['dur_ms']>10] #5ms

emp=[]
All_SWR = pd.DataFrame(emp, columns = ['start time', 'end time', 'Duration'])
All_SWR['start time']=rip_ep['start']*1000
All_SWR['end time']=rip_ep['end']*1000
All_SWR['Duration']=rip_ep['dur_ms']
All_SWR['toKeep']= 'True'

mingap=1 #in seconds
prevSwr=[]
for swr in All_SWR.index :
    start=All_SWR.loc[swr, "start time"]
    previousend=All_SWR.loc[prevSwr, "end time"] if prevSwr else start-mingap*1000                             
    prevSwr=swr      

    if start - previousend >= mingap*1000:
        All_SWR.loc[swr, "Overlapping_Baseline"] = 'False'
    else:
        All_SWR.loc[swr, "Overlapping_Baseline"] = 'True'

# Store the results in All_SWR_prop pd dataframe and save as pkl/csv for post processing.
filename = folder_base / f'SWR_finedetection.csv'
All_SWR.to_csv(filename)

print(len(All_SWR), 'SWR detected in CA1')

PFC

In [ ]:
lfp = nap.Tsd(t=np.arange(len(PFC))/samplerate, d=PFC)
freqs = np.geomspace(3, 200, 100)
mwt_RUN = nap.compute_wavelet_transform(lfp, fs=samplerate, freqs=freqs, gaussian_width=3, window_length=1.5)
spdl_freq_index = np.logical_and(freqs > 11, freqs < 17)
spdl_power = np.mean(np.abs(mwt_RUN[:, spdl_freq_index]), 1)
smoothed_spdl_power = spdl_power.smooth(0.1) #in seconds 0.005 

In [ ]:
winlen = 30 # default window length in sec

#you must first create a main Qt application (for event loop)
app = mkQApp()

t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer(debug=False, show_auto_scale=True)

#create a viewer for signal
combined = np.stack([zscore(lfp.as_array())/50, smoothed_spdl_power.as_array(), np.linspace(thPFC, thPFC,len(lfp.as_array())) ], axis=1)
source =InMemoryAnalogSignalSource(combined, samplerate, t_start)
view1 = TraceViewer(source=source)

view1.params['xsize']= winlen
view1.params['display_labels'] = True
#view1.params['scale_mode'] = 'same_for_all'
colormap = np.insert(['#88FF88', '#8888FF', '#FF8888']* 10, 0, '#FFFFFF')
view1.by_channel_params[f'ch1', 'color'] = colormap[0]
view1.by_channel_params[f'ch0', 'color'] = colormap[10]
view1.by_channel_params[f'ch0', 'name'] = 'PFC'
view1.by_channel_params[f'ch1', 'name'] = 'filtered PFC'
view1.auto_scale()


# FFT
view3 = TimeFreqViewer(source=source, name='FFT')

view3.params['show_axis'] = True
view3.params['timefreq', 'f_start'] = 1
view3.params['timefreq', 'f_stop'] = 30
view3.params['timefreq', 'deltafreq'] = 1 #interval in Hz
view3.params['xsize'] = winlen
view3.by_channel_params[f'ch0', 'clim'] = .0500
view3.by_channel_params[f'ch0', 'visible'] = True

#show main window and run Qapp

win.add_view(view1)
win.add_view(view3)
win.navigation_toolbar.spinbox_xsize.setValue(winlen)
win.show()

app.exec()

# press '1', '2', '3', '4' etc, to encode state.
# or toggle 'Time range selector' and then use 'Insert within range'

In [ ]:
thPFC=.4
threshold_spdl_power = smoothed_spdl_power.threshold(thPFC) #.4 is great
spdl_ep = threshold_spdl_power.time_support
spdl_ep['dur_ms']=np.round((spdl_ep['end']-spdl_ep['start'])*1000)
spdl_ep=spdl_ep[spdl_ep['dur_ms']>500]

emp=[]
All_SpindlePFC = pd.DataFrame(emp, columns = ['start time', 'end time', 'Duration'])
All_SpindlePFC['start time']=spdl_ep['start']*1000
All_SpindlePFC['end time']=spdl_ep['end']*1000
All_SpindlePFC['Duration']=spdl_ep['dur_ms']
All_SpindlePFC['toKeep']= 'True'

filename = folder_base / f'SpindlesPFC_finedetection.csv'
All_SpindlePFC.to_csv(filename)

print(len(All_SpindlePFC), 'Spdl detected in PFC')

PFCspdl_start = np.transpose(All_SpindlePFC['start time'].astype(int))
PFCspdl_end = np.transpose(All_SpindlePFC['end time'].astype(int))


In [ ]:
winlen = 30

app = mkQApp()
win = MainViewer(debug=False, show_auto_scale=True)

# LFPs
try: 
    scatter_indexes = {0: PFCspdl_start, 1: PFCspdl_end}
    scatter_channels = {0: [0], 1: [0]}
except: 
    scatter_indexes={}
    scatter_channels={}

#Create the main window that can contain several viewers
win = MainViewer(debug=False, show_auto_scale=True)
t_start=0.
#create a viewer for signal
source = AnalogSignalSourceWithScatter(combined, samplerate, t_start, scatter_indexes, scatter_channels, scatter_colors= {0: '#FFFFFF', 1: '#222222'})
view1 = TraceViewer(source=source)

view1.params['xsize']= winlen
view1.params['display_labels'] = True
#view1.params['scale_mode'] = 'same_for_all'
colormap = np.insert(['#88FF88', '#8888FF', '#FF8888']* 10, 0, '#FFFFFF')
view1.by_channel_params[f'ch1', 'color'] = colormap[0]
view1.by_channel_params[f'ch0', 'color'] = colormap[10]
view1.by_channel_params[f'ch0', 'name'] = 'PFC'
view1.by_channel_params[f'ch1', 'name'] = 'filtered PFC'
view1.auto_scale()


# FFT
view3 = TimeFreqViewer(source=source, name='FFT')

view3.params['show_axis'] = True
view3.params['timefreq', 'f_start'] = 1
view3.params['timefreq', 'f_stop'] = 30
view3.params['timefreq', 'deltafreq'] = 1 #interval in Hz
view3.params['xsize'] = winlen
view3.by_channel_params[f'ch0', 'clim'] = .0500
view3.by_channel_params[f'ch0', 'visible'] = True

#show main window and run Qapp

win.add_view(view1)
win.add_view(view3)
win.navigation_toolbar.spinbox_xsize.setValue(winlen)
win.show()

app.exec()

# press '1', '2', '3', '4' etc, to encode state.
# or toggle 'Time range selector' and then use 'Insert within range'

S1

In [ ]:
lfp = nap.Tsd(t=np.arange(len(S1))/samplerate, d=S1)
freqs = np.geomspace(3, 200, 100)
mwt_RUN = nap.compute_wavelet_transform(lfp, fs=samplerate, freqs=freqs, gaussian_width=3, window_length=1.5)
spdl_freq_index = np.logical_and(freqs > 11, freqs < 17)
spdl_power = np.mean(np.abs(mwt_RUN[:, spdl_freq_index]), 1)
smoothed_spdl_power = spdl_power.smooth(0.1) #in seconds 0.005 

In [ ]:
winlen = 30 # default window length in sec

#you must first create a main Qt application (for event loop)
app = mkQApp()

t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer(debug=False, show_auto_scale=True)

#create a viewer for signal
combined = np.stack([zscore(lfp.as_array())/50, smoothed_spdl_power.as_array(), np.linspace(thS1, thS1,len(lfp.as_array())) ], axis=1)
source =InMemoryAnalogSignalSource(combined, samplerate, t_start)
view1 = TraceViewer(source=source)

view1.params['xsize']= winlen
view1.params['display_labels'] = True
#view1.params['scale_mode'] = 'same_for_all'
colormap = np.insert(['#88FF88', '#8888FF', '#FF8888']* 10, 0, '#FFFFFF')
view1.by_channel_params[f'ch1', 'color'] = colormap[0]
view1.by_channel_params[f'ch0', 'color'] = colormap[10]
view1.auto_scale()


# FFT
view3 = TimeFreqViewer(source=source, name='FFT')

view3.params['show_axis'] = True
view3.params['timefreq', 'f_start'] = 1
view3.params['timefreq', 'f_stop'] = 30
view3.params['timefreq', 'deltafreq'] = 1 #interval in Hz
view3.params['xsize'] = winlen
view3.by_channel_params[f'ch0', 'clim'] = .0500
view3.by_channel_params[f'ch0', 'visible'] = True

#show main window and run Qapp

win.add_view(view1)
win.add_view(view3)
win.navigation_toolbar.spinbox_xsize.setValue(winlen)
win.show()

app.exec()

# press '1', '2', '3', '4' etc, to encode state.
# or toggle 'Time range selector' and then use 'Insert within range'

In [ ]:
thS1=.4
threshold_spdl_power = smoothed_spdl_power.threshold(thS1) #.4 is great
spdl_ep = threshold_spdl_power.time_support
spdl_ep['dur_ms']=np.round((spdl_ep['end']-spdl_ep['start'])*1000)
spdl_ep=spdl_ep[spdl_ep['dur_ms']>500]

emp=[]
All_SpindleS1 = pd.DataFrame(emp, columns = ['start time', 'end time', 'Duration'])
All_SpindleS1['start time']=spdl_ep['start']*1000
All_SpindleS1['end time']=spdl_ep['end']*1000
All_SpindleS1['Duration']=spdl_ep['dur_ms']
All_SpindleS1['toKeep']= 'True'

filename = folder_base / f'SpindlesS1_finedetection.csv'
All_SpindleS1.to_csv(filename)

print(len(All_SpindleS1), 'Spdl detected in S1')

S1spdl_start = np.transpose(All_SpindleS1['start time'].astype(int))
S1spdl_end = np.transpose(All_SpindleS1['end time'].astype(int))

In [ ]:
winlen = 30

app = mkQApp()
win = MainViewer(debug=False, show_auto_scale=True)

# LFPs
try: 
    scatter_indexes = {0: S1spdl_start, 1: S1spdl_end}
    scatter_channels = {0: [0], 1: [0]}
except: 
    scatter_indexes={}
    scatter_channels={}

#Create the main window that can contain several viewers
win = MainViewer(debug=False, show_auto_scale=True)
t_start=0.
#create a viewer for signal
source = AnalogSignalSourceWithScatter(combined, samplerate, t_start, scatter_indexes, scatter_channels, scatter_colors= {0: '#FFFFFF', 1: '#222222'})
view1 = TraceViewer(source=source)

view1.params['xsize']= winlen
view1.params['display_labels'] = True
#view1.params['scale_mode'] = 'same_for_all'
colormap = np.insert(['#88FF88', '#8888FF', '#FF8888']* 10, 0, '#FFFFFF')
view1.by_channel_params[f'ch1', 'color'] = colormap[0]
view1.by_channel_params[f'ch0', 'color'] = colormap[10]
view1.by_channel_params[f'ch0', 'name'] = 'PFC'
view1.by_channel_params[f'ch1', 'name'] = 'filtered PFC'
view1.auto_scale()


# FFT
view3 = TimeFreqViewer(source=source, name='FFT')

view3.params['show_axis'] = True
view3.params['timefreq', 'f_start'] = 1
view3.params['timefreq', 'f_stop'] = 30
view3.params['timefreq', 'deltafreq'] = 1 #interval in Hz
view3.params['xsize'] = winlen
view3.by_channel_params[f'ch0', 'clim'] = .0500
view3.by_channel_params[f'ch0', 'visible'] = True

#show main window and run Qapp

win.add_view(view1)
win.add_view(view3)
win.navigation_toolbar.spinbox_xsize.setValue(winlen)
win.show()

app.exec()

# press '1', '2', '3', '4' etc, to encode state.
# or toggle 'Time range selector' and then use 'Insert within range'

Save threshold

In [ ]:
names = ['CA1_SWR', 'PFC_spdl', 'S1_spdl']
values = [thCA1, thPFC, thS1]

with open(f"{dpath}/Osc_finedetection_thresholds.txt", "w") as f:
    for name, val in zip(names, values):
        f.write(f"{name}: {val}\n")

Detect global spindles

In [ ]:
All_SpindleS1 = pd.read_csv(Path(f'{folder_base}\SpindlesS1_finedetection.csv'))
All_SpindlePFC = pd.read_csv(Path(f'{folder_base}\SpindlesPFC_finedetection.csv'))

All_SpindlePFC['CTX']='PFC'
All_SpindlePFC['StartingLoc']='PFC'
All_SpindlePFC = All_SpindlePFC.reset_index(drop=True)
#ewPFClist=restriction_parameter(All_SpindlePFC)
NewPFClist=All_SpindlePFC

All_SpindleS1['CTX']='S1'
All_SpindleS1['StartingLoc']='S1'
All_SpindleS1 = All_SpindleS1.reset_index(drop=True)
#NewS1list=restriction_parameter(All_SpindleS1)
NewS1list=All_SpindleS1

#Spdllist=pd.concat([All_SpindlePFC,All_SpindleS1],ignore_index=False)  
Spdllist=pd.concat([NewPFClist,NewS1list],ignore_index=False)  

Spdllist['LocalGlobal']='Local'
Spdllist['DistanceClosestSpdl']=np.inf

Spdllist = Spdllist.sort_values(by='start time')
Spdllist = Spdllist.reset_index(drop=True)

liststarts=Spdllist["start time"]
listends=Spdllist["end time"]
NewSpdllist=Spdllist.copy()
NewSpdllist['toKeep'] = NewSpdllist['toKeep'].astype(str)

for spdl1 in NewSpdllist.index : # range(len(Spdllist)) :
    start1=liststarts[spdl1]
    end1=listends[spdl1]
    otherunit_range = [x for x in NewSpdllist.index  if x != spdl1]
    #print(f'spdl1 n°{spdl1}')
    if NewSpdllist.loc[spdl1,'toKeep'] != 'False':
        for spdl2 in otherunit_range:
            if NewSpdllist.loc[spdl2,'toKeep'] != 'False':
                start2=liststarts[spdl2]
                end2=listends[spdl2]
                if NewSpdllist.loc[spdl1,'CTX']!= NewSpdllist.loc[spdl2,'CTX']: # Needs to be from 2 differents brain areas

                    if start2>start1:
                        NewSpdllist.loc[spdl1, 'DistanceClosestSpdl'] = start1-start2 if start2-start1<NewSpdllist.loc[spdl1, 'DistanceClosestSpdl'] else NewSpdllist.loc[spdl1, 'DistanceClosestSpdl']
                        NewSpdllist.loc[spdl2, 'DistanceClosestSpdl'] = start2-start1 if start2-start1<NewSpdllist.loc[spdl2, 'DistanceClosestSpdl'] else NewSpdllist.loc[spdl2, 'DistanceClosestSpdl']
                    else:
                        NewSpdllist.loc[spdl1, 'DistanceClosestSpdl'] = start1-start2 if start1-start2<NewSpdllist.loc[spdl1, 'DistanceClosestSpdl'] else NewSpdllist.loc[spdl1, 'DistanceClosestSpdl']
                        NewSpdllist.loc[spdl2, 'DistanceClosestSpdl'] = start2-start1 if start1-start2<NewSpdllist.loc[spdl2, 'DistanceClosestSpdl'] else NewSpdllist.loc[spdl2, 'DistanceClosestSpdl']

                    if start1<=start2 and start2<=end1: # event n°2 begins after the start n°1               
                        if (end1-start2)>=int(0.5*(end1-start1)): # overlapp > to 50% of the duration of the event n°1                                
                            NewSpdllist.loc[spdl1, 'LocalGlobal']='Global'
                            NewSpdllist.loc[spdl1, 'StartingLoc']=NewSpdllist.loc[spdl1,'StartingLoc']
                            NewSpdllist.loc[spdl1, 'CTX']='S1PFC'
                            NewSpdllist.loc[spdl1, 'start time']=int((start1 + start2)/2)
                            NewSpdllist.loc[spdl1, 'end time']=max(end1, end2)
                            NewSpdllist.loc[spdl1, 'Duration']=max(end1, end2)-int((start1 + start2)/2)
                            NewSpdllist.loc[spdl2, 'toKeep']='False'
                            #print(f'Cdt n°1: Global, keep spdl n°{spdl1} {start1} instead of spdl n°{spdl2} {start2}')
                            break
                    elif start1<=end2 and end2<=end1: # event n°2 ends before the end n°1 
                        if (end2-start1)>=int(0.5*(end1-start1)): # overlapp > to 50% of the duration of the event n°1
                            NewSpdllist.loc[spdl2, 'LocalGlobal']='Global'
                            NewSpdllist.loc[spdl2, 'StartingLoc']=NewSpdllist.loc[spdl2,'StartingLoc']
                            NewSpdllist.loc[spdl2, 'CTX']='S1PFC'
                            NewSpdllist.loc[spdl2, 'start time']=int((start1 + start2)/2)
                            NewSpdllist.loc[spdl2, 'end time']=max(end1, end2)
                            NewSpdllist.loc[spdl2, 'Duration']=max(end1, end2)-int((start1 + start2)/2)
                            NewSpdllist.loc[spdl1, 'toKeep']='False'
                            #print(f'Cdt n°2: Global, keep spdl n°{spdl2} {start2}, instead of spdl n°{spdl1} {start1}')
                            break
NewSpdllist = NewSpdllist[NewSpdllist['toKeep'].isin(['True', 'VRAI'])]
NewSpdllist = NewSpdllist.sort_values(by='start time')
NewSpdllist = NewSpdllist.reset_index(drop=True)
NewSpdllist['DistanceClosestSpdl'] = NewSpdllist['DistanceClosestSpdl'] *-1 # to have in positive the spdl that arrives after the onset and vice versa
#NewSpdllist=restriction_parameter(NewSpdllist)

print(len(NewSpdllist[NewSpdllist['CTX']=='S1']), 'Spdl detected in S1')
print(len(NewSpdllist[NewSpdllist['CTX']=='PFC']), 'Spdl detected in PFC')
print(len(NewSpdllist[NewSpdllist['CTX']=='S1PFC']), 'Spdl detected in S1PFC')

mingap=1 #in seconds
prevSpdl=[]
for spdl in NewSpdllist.index :
    start=NewSpdllist.loc[spdl, "start time"]
    previousend=NewSpdllist.loc[prevSpdl, "end time"] if prevSpdl else start-mingap*1000                             
    prevSpdl=spdl       

    if start - previousend >= mingap*1000:
        NewSpdllist.loc[spdl, "Overlapping_Baseline"] = 'False'
    else:
        NewSpdllist.loc[spdl, "Overlapping_Baseline"] = 'True'

filenameOutput = folder_base / f'SpindlesS1&PFC_finedetection.csv' 
NewSpdllist.to_csv(filenameOutput, sep= ',')

Option: see all detected oscillations

In [ ]:
# Load detected oscillations timestamps
# SWR     
SWSdetection=folder_base / f'SWR_finedetection.csv'
SWR_prop = pd.read_csv(SWSdetection, index_col=0)  
SWR_prop['toKeep'] = SWR_prop['toKeep'].astype(str)
SWR_prop  = SWR_prop[SWR_prop['toKeep'].isin(['VRAI', 'True'])]   
SWR_start = np.transpose((SWR_prop['start time']).astype(int))
SWR_end = np.transpose((SWR_prop['end time']).astype(int))

# Spindles
Spdldetection=folder_base / f'SpindlesS1&PFC_finedetection.csv'
Spdl_prop = pd.read_csv(Spdldetection, index_col=0)  
Spdl_prop['toKeep'] = Spdl_prop['toKeep'].astype(str)
Spdl_prop  = Spdl_prop[Spdl_prop['toKeep'].isin(['VRAI', 'True'])]   

S1Spdl_prop= Spdl_prop[Spdl_prop['CTX']=='S1']
S1spdl_start = np.transpose(S1Spdl_prop['start time'].astype(int))
S1spdl_end = np.transpose(S1Spdl_prop['end time'].astype(int))

PFCSpdl_prop= Spdl_prop[Spdl_prop['CTX']=='PFC']
PFCspdl_start = np.transpose(PFCSpdl_prop['start time'].astype(int))
PFCspdl_end = np.transpose(PFCSpdl_prop['end time'].astype(int))

GlobalSpdl_prop= Spdl_prop[Spdl_prop['CTX']=='S1PFC']
GBLspdl_start = np.transpose(GlobalSpdl_prop['start time'].astype(int))
GBLspdl_end = np.transpose(GlobalSpdl_prop['end time'].astype(int))

print(f'{len(SWR_end)} SWR in CA1, {len(S1spdl_end)} Spdl in S1, {len(PFCspdl_end)} Spdl in PFC, {len(GBLspdl_end)} Global Spdl')

In [ ]:
labels = ['AW_1', 'QW_2', 'NREM_3', 'IS_4',  'REM_5', 'undefined_6'] # Active wake, Quiet wake, NREM sleep, Intermediate sleep, REM sleep, undefined states
epoch_dur = 5 # define epoch duration in sec
winlen = 30

app = mkQApp()
win = MainViewer(debug=False, show_auto_scale=True)

# LFPs
try: 
    scatter_indexes = {0: PFCspdl_start, 1: PFCspdl_end, 2: S1spdl_start, 3: S1spdl_end, 4: GBLspdl_start, 5: GBLspdl_end, 6: SWR_start, 7: SWR_end}
    scatter_channels = {0: [2], 1: [2], 2: [1], 3: [1], 4: [1,2], 5: [1,2], 6: [3], 7: [3]}
except: 
    scatter_indexes={}
    scatter_channels={}

#Create the main window that can contain several viewers
t_start=0.
#create a viewer for signal
source = AnalogSignalSourceWithScatter(combined0, samplerate, t_start, scatter_indexes, scatter_channels, scatter_colors= {0: '#FFFFFF', 1: '#222222', 2: '#FFFFFF', 3: '#222222', 4: '#BBBBBB', 5: '#555555', 6: '#FFFFFF', 7: '#222222'}, channel_names=RecordedArea)
view1 = TraceViewer(source=source)

view1.params['xsize']= winlen
view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
colormap = np.insert(['#88FF88', '#8888FF', '#FF8888']* 10, 0, '#FFFFFF')
for idx, ch in enumerate(RecordedArea): 
    view1.by_channel_params[f'ch{idx}', 'color'] = colormap[idx] #FF0000 red, #00FF00 green, and #0000FF blue
view1.auto_scale()


# FFT
view3 = TimeFreqViewer(source=source, name='FFT')

view3.params['show_axis'] = True
view3.params['timefreq', 'f_start'] = 1
view3.params['timefreq', 'f_stop'] = 30
view3.params['timefreq', 'deltafreq'] = 1 #interval in Hz
view3.params['xsize'] = winlen
for idx, ch in enumerate(RecordedArea):
    if ch == 'EMG': 
        view3.by_channel_params[f'ch{idx}', 'visible'] = False
    else:        
        view3.by_channel_params[f'ch{idx}', 'clim'] = 1
        view3.by_channel_params[f'ch{idx}', 'visible'] = True


#add them to mainwindow

win.add_view(view1)
win.add_view(view3)

#Run

win.show()
app.exec_()